# 영화추천 시스템

## 유사도를 이용한 추천 시스템

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
movie = pd.read_csv("data/movies_metadata.csv", low_memory=False)
movie.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [3]:
movie.shape

(45466, 24)

In [4]:
df = movie[['title','overview']]
df.head(2)

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...


In [5]:
df.overview[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

## 데이터 전처리

In [6]:
df.isnull().sum()

title         6
overview    954
dtype: int64

In [7]:
df.dropna(inplace=True)     # how='any' 가 디폴트
df.shape

(44506, 2)

In [8]:
# data는 삭제되었지만 남은 데이터의 index 번호는 그대로 남아있음
df.tail()

,title,overview
45461,Subdue,Rising and falling between a man and woman.
45462,Century of Birthing,An artist struggles to finish his work while a...
45463,Betrayal,"When one of her hits goes wrong, a professiona..."
45464,Satan Triumphant,"In a small town live two brothers, one a minis..."
45465,Queerama,50 years after decriminalisation of homosexual...


In [9]:
df.set_index('title', inplace=True)
df.reset_index(inplace=True)
df.tail()

,title,overview
44501,Subdue,Rising and falling between a man and woman.
44502,Century of Birthing,An artist struggles to finish his work while a...
44503,Betrayal,"When one of her hits goes wrong, a professiona..."
44504,Satan Triumphant,"In a small town live two brothers, one a minis..."
44505,Queerama,50 years after decriminalisation of homosexual...


## 텍스트 전처리

In [10]:
# 구둣점 제거
df['clean_doc'] = df.overview.str.replace('[^A-Za-z ]', '')
df.head(3)

,title,overview,clean_doc
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Led by Woody Andys toys live happily in his ro...
1,Jumanji,When siblings Judy and Peter discover an encha...,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,A family wedding reignites the ancient feud be...


## DTM 변환

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(stop_words='english')
tfidf_matrix = tvect.fit_transform(df.overview)
tfidf_matrix.shape

(44506, 75827)

In [12]:
tfidf_clean = tvect.fit_transform(df.clean_doc)
tfidf_clean.shape

(44506, 88616)

## 영화의 타이틀과 인덱스를 가진 테이블

In [13]:
indices = pd.Series(df.index, index=df.title).drop_duplicates()
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [14]:
indices['Jumanji']

1

## 코사인 유사도를 통해 유사한 영화를 찾는 함수

In [15]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_clean = linear_kernel(tfidf_clean, tfidf_clean)

MemoryError: Unable to allocate 14.8 GiB for an array with shape (44506, 44506) and data type float64

In [16]:
cosine_sim.shape

(44506, 44506)

In [17]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 구함
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return df.title.iloc[movie_indices]

In [18]:
# 텍스트 전처리를 하지 않은 데이터
get_recommendations('The Dark Knight Rises')

12447                                      The Dark Knight
149                                         Batman Forever
1314                                        Batman Returns
15444                           Batman: Under the Red Hood
583                                                 Batman
21036    Batman Unmasked: The Psychology of the Dark Kn...
9203                    Batman Beyond: Return of the Joker
17930                                     Batman: Year One
19661              Batman: The Dark Knight Returns, Part 1
3077                          Batman: Mask of the Phantasm
Name: title, dtype: object

In [19]:
# 텍스트 전처리를 한 데이터
get_recommendations('The Dark Knight Rises', cosine_clean)

NameError: name 'cosine_clean' is not defined